In [204]:
import numpy             as np
import matplotlib.pyplot as plt
import smtplib
import ipywidgets      as widgets
from   ipywidgets      import Button, HBox, VBox
from   IPython.display import display
import os

from   IPython.display import clear_output
from IPython.display import display, HTML
from IPython.display import FileLink, FileLinks
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 



count = 0

title = widgets.HTML(
        value="<b>Image Survey</b>",
        placeholder='',
        description='',
    )

instructions = widgets.HTML(
        value="Instructions for survey:",
        placeholder='',
        description='',
    )

i1 = widgets.HTML(
        value="There will be displayed an image in front of you. You must go through each category below the image and choose the probability that the image may be the category given. ",
        placeholder='',
        description='',
    )
i2 = widgets.HTML(
        value="Give a rating on how sure you are about the image belonging to each category:",
        placeholder='',
        description='',
    )
r0 = widgets.HTML(
        value="0 = There is no way this image falls under this category and there are no details of this in the image",
        placeholder='',
        description='',
    )
r1 = widgets.HTML(
        value="1 = There's a small chance that the image is part of this category",
        placeholder='',
        description='',
    )
r2 = widgets.HTML(
        value="2 = The image might be in this category, but definitely can be in another category too",
        placeholder='',
        description='',
    )
r3 = widgets.HTML(
        value="3 = It's probably this category, but there are details that can be interpreted as something else",
        placeholder='',
        description='',
    )
r4 = widgets.HTML(
        value="4 = You can say with 100% certainty that the photo definitely belongs to this category",
        placeholder='',
        description='',
    )
r_0 = widgets.HTML(
        value="0 = There is no way this image falls under this category and there are no details of this in the image",
        placeholder='',
        description='',
    )
r_1 = widgets.HTML(
        value="1 = There's a small chance that the image is part of this category",
        placeholder='',
        description='',
    )
r_2 = widgets.HTML(
        value="2 = The image might be in this category, but definitely can be in another category too",
        placeholder='',
        description='',
    )
r_3 = widgets.HTML(
        value="3 = It's probably this category, but there are details that can be interpreted as something else",
        placeholder='',
        description='',
    )
r_4 = widgets.HTML(
        value="4 = You can say with 100% certainty that the photo definitely belongs to this category",
        placeholder='',
        description='',
    )
i3 = widgets.HTML(
        value="Try to choose at least 3 categories to be non-zero.",
        placeholder='',
        description='',
    )
i4 = widgets.HTML(
        value="You will receive around 200 photos to look at and rate how much they look like each object. This should take around an hour. Once you are done rating the photos, the survey will naturally finish. If you do not do not have time to complete rating the photos, press the button done!. DO NOT press done! if you are not finished, as you will have to restart your work. Once you are done, you must email Ophir the results. Let Ophir know if you have any questions.",
        placeholder='',
        description='',
    )


#Importing all files needed for this notebook


filename   = 'data_batch_1'
filename2 = 'Batch Names-Copy1'

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

file = unpickle(filename)
file2 = unpickle(filename2)

images = file[b'data'] 
labels = file[b'labels']
label_names = file2[b'label_names']

#loading all file data onto notebook

images = images.reshape(len(labels), 3, 32, 32)

def imagshow(img):
    plt.imshow(np.transpose(img, (1, 2, 0))) #flip the grid from side to normal
    plt.show()
# plotting and showing the images


dataset = {}
total = len(images)
image_ids = np.arange(total)
images_per_person = 200
people = int(total/images_per_person)
for I in range(people):
    ind1 = I*images_per_person
    ind2 = (I+1)*images_per_person
    dataset[I] = images[ind1:ind2,:,:,:]
results= {}    
I = 0
answers = []
datalen = len(labels)

#arranging all the data to be used    

def user(b):
    display(widgets.VBox([name, userid, submit]))
    instructions.close()
    title.close()
    i1.close()
    r0.close()
    r1.close()
    r2.close()
    r3.close()
    r4.close()
    i2.close()
    i3.close()
    i4.close()
    ok.close()

def user_id_images(b):
    display(r_4)
    display(r_3)
    display(r_2)
    display(r_1)
    display(r_0)
    global count 
    try: 
        V = userid.value
        images = dataset[V]
        invalid.close()
        with out:
            imagshow(images[V])
        display(out)
        display(widgets.VBox([widgetsBox, buttonNext]))
        display(widgets.VBox([done]))
        userid.close()
        submit.close()
        name.close()
    except:
        if count == 0:
            display(invalid)
            count+=1
        else:
            count+=1
        
#Displaying all of the data        
    
    
def on_buttonNext_clicked(b):
    R = userid.value
    V1 = userid.value
    images = dataset[R]
    global V
    #Only changes I for individual actions
    V = V+1
    if V < images_per_person:
        with out: 
        #Do the things within "out" - which is the widget output
            row = []
        #The newlist that the answers will be appended to
            for l in range(0,len(widget_labels)): #Creating widget answers by changing the individual widget values
                row.extend([items[l].value]) #appending the widget answers 
                items[l].value = 0 #Resets the values
            answers.append(row) #Puts in the new row answers into the overall answers
            ID = (images_per_person*R)+V
            results[V] = ID, row, labels[ID-1]
            clear_output() #Resets the widget numbers
            imagshow(images[V]) #Displays a new image 
            images_done = widgets.HTML(
                value= str(len(results)) + "/" + str(images_per_person),
                placeholder='',
                description='',
            )
            display(images_done)
    else:
        finished_survey(b)

        
#Moving onto the next question



dataresults = 'dataresults.npz'
def finished_survey(b):
    r_4.close()
    r_3.close()
    r_2.close()
    r_1.close()
    r_0.close()
    N = name.value
    V1 = userid.value
    ID1= (images_per_person*V1)
    ID2 = len(results)+ID1
    widgetsBox.close()
    buttonNext.close()
    out.close()
    done.close()
    print("""Thank you! Please copy and paste the following results and send to cifar10surveyanswers@gmail.com
    
    
    To: cifar10surveyanswers@gmail.com
    
    Subject: {N}({V1}):{ID1}-{ID2}
    
    Body: {results}
    
    """.format(N=str(N),V1=str(V1),ID1=str(ID1),ID2=str(ID2), results=str(results))) 




In [ ]:

# # Image Survey

# ### Instructions for the survey:

# There will be displayed an image in front of you. You must go through each category below the image and choose the probability that the image may be the category given. 

# Give a rating on how sure you are about the image belonging to each category:

# 4 = You can say with 100% certainty that the photo definitely belongs to this category <br>
# 3 = It's probably this category, but there are details that can be interpreted as something else <br>
# 2 = The image might be in this category, but definitely can be in another category too <br>
# 1 = There's a small chance that the image is part of this category <br> 
# 0 = There is no way this image falls under this category and there are no details of this in the image <br>

# Try to choose at least 3 categories to be non-zero. 

# You will receive around 200 photos to look at and rate how much they look like each object. This should take around an hour. Once you are done rating the photos, the survey will naturally finish. If you do not do not have time to complete rating the photos, press the button "done!". DO NOT press "done" if you are not finished, as you will have to restart your work. Once you are done, you must email Ophir the results. Let Ophir know if you have any questions. 


In [205]:
widget_labels = ['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
confidence    = [4, 3, 2, 1, 0]
items  = [widgets.RadioButtons(options=confidence, 
                layout=widgets.Layout(width='50px', height='150px'), value = 0) for l in widget_labels] 
widgetsBox = widgets.HBox([widgets.VBox([widgets.Label(widget_labels[l]), items[l]]) for l in range(0,len(widget_labels))])
out        = widgets.Output()
#Base of files 

userid = widgets.IntText(
    value= 0,
    description='USER ID#:',
    disabled=False)

invalid = widgets.HTML(
    value="Invalid USER ID",
    placeholder='',
    description='',
)

ok = widgets.Button(description='OK')

name = widgets.Text(
    value='',
    placeholder=' ',
    description='Full Name:',
    disabled=False
)


display(title)
display(instructions)
display(i1)
display(i2)
display(r4)
display(r3)
display(r2)
display(r1)
display(r0)
display(i3)
display(i4)
display(ok)

ok.on_click(user)

#Interactive widgets
buttonNext = widgets.Button(description='Next')
submit = widgets.Button(description="Submit")
back = widgets.Button(description="Back")
done = widgets.Button(description= 'Done!')
#buttons

buttonNext.on_click(on_buttonNext_clicked)
done.on_click(finished_survey)
    
submit.on_click(user_id_images)

#Displaying the original userid box


V = userid.value
ID1= (images_per_person*V)

HTML(value='<b>Image Survey</b>', placeholder='')

HTML(value='Instructions for survey:', placeholder='')

HTML(value='There will be displayed an image in front of you. You must go through each category below the imag…

HTML(value='Give a rating on how sure you are about the image belonging to each category:', placeholder='')

HTML(value='4 = You can say with 100% certainty that the photo definitely belongs to this category', placehold…

...

HTML(value='2 = The image might be in this category, but definitely can be in another category too', placehold…

HTML(value="1 = There's a small chance that the image is part of this category", placeholder='')

HTML(value='0 = There is no way this image falls under this category and there are no details of this in the i…

Output()

Thank you! Please copy and paste the following results and send to cifar10surveyanswers@gmail.com
    
    
    To: cifar10surveyanswers@gmail.com
    
    Subject: Ophir Greif(3):600-609
    
    Body: {1: (601, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0), 2: (602, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 8), 3: (603, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 8), 4: (604, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3), 5: (605, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0), 6: (606, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0), 7: (607, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1), 8: (608, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 5), 9: (609, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0)}
    
    


In [202]:
# import numpy as np
# def softmax(v,theta=.1):
#     v = np.float32(v)
#     t = np.exp(theta*v)
#     t = t/np.sum(t)
#     return t
    

In [203]:
# r = unpickle('Ophir Greif(1)_200-399')
# r

In [155]:

#t = softmax(v,1)
#np.sum(t)
#print(t)

In [201]:
# def getConfusionMatrix(R):
    
#     C = np.zeros((10,10),dtype=float)
#     n = len(R)
#     d = np.zeros(10,dtype=float)
#     for i in range(n):
#         v = R[i+201][0]
#         t = softmax(v,1)
#         ind = R[i+201][1]
#         C[ind,:] += t
#         d[ind] += 1.0
        
#     for i in range(10):
#         if d[i] > 0:
#             C[i,:] = C[i,:]/d[i]
            
#     return C, d        

In [200]:
# import matplotlib.pyplot as plt
# C,d = getConfusionMatrix(r)
# plt.imshow(C)
# plt.colorbar()

In [198]:
# softmax([0,0,0,0,0,0,0,0,0,5],1)

In [199]:
# def getConfusionMatrix(R):
    
#     C = np.zeros((10,10),dtype=float)
#     n = len(R)
#     d = np.zeros(10,dtype=float)
#     for i in range(n):
#         v = R[i+1][1]
#         t = softmax(v,1.0)
#         ind = R[i+1][2]
#         C[ind,:] += t
#         d[ind] += 1.0
        
#     for i in range(10):
#         if d[i] > 0:
#             C[i,:] = C[i,:]/d[i]
            
#     return C, d        